In [2]:
import os
import numpy as np

import torch
torch.manual_seed(29)
from torch import nn
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import torch.nn.parallel
cudnn.benchmark = True
from torch.utils.data import DataLoader

from torchstat import stat
from utils import  models
from utils.data import dataset_1
from utils.trainer_utils import parfilter

from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from skimage.measure import compare_psnr, compare_ssim

from matplotlib import pyplot as plt

from glob import glob
from tqdm import tqdm

from PIL.PngImagePlugin import PngImageFile, PngInfo

In [2]:
def image_quality_comp(I, Iout, Enout):
    
    for x in ['test.png', 'test_35.jpg', 'test.flif', 'test1.png', 'test1.flif', 'test_comp.png']: 
        if os.path.exists(x):os.remove(x)
    
    I.save('test.png')
    Iout.save('test_comp.png')
    I.save('test_10.jpg', quality = 10)
    Enout.save('test1.png')
    
    os.system("flif -e test.png test.flif")
    os.system("flif -e test1.png test1.flif")
    
    print('Original Image :: ')
    print('PNG     :: ' + str(8*os.path.getsize('test.png')/(I.size[0] * I.size[1])))
    print('JPG-10% :: ' + str(8*os.path.getsize('test_10.jpg')/(I.size[0] * I.size[1])))
    print('FLIF    :: ' + str(8*os.path.getsize('test.flif')/(I.size[0] * I.size[1])))
    print('Encoded Image :: ')
    print('PNG     :: ' + str(8*os.path.getsize('test1.png')/(I.size[0] * I.size[1])))
    print('FLIF    :: ' + str(8*os.path.getsize('test1.flif')/(I.size[0] * I.size[1])))
    
def image_quality_comp2(I, Iout):
    
    for x in ['test.png', 'test_35.jpg', 'test.flif', 'test1.png', 'test1.flif', 'test_comp.png']: 
        if os.path.exists(x):os.remove(x)
    
    I.save('test.png')
    Iout.save('test_comp.png')
    I.save('test_10.jpg', quality = 10)
    
    os.system("flif -e test.png test.flif")
    
    print('Original Image :: ')
    print('PNG     :: ' + str(8*os.path.getsize('test.png')/(I.size[0] * I.size[1])))
    print('JPG-10% :: ' + str(8*os.path.getsize('test_10.jpg')/(I.size[0] * I.size[1])))
    print('FLIF    :: ' + str(8*os.path.getsize('test.flif')/(I.size[0] * I.size[1])))

In [3]:
def compress_and_decompress(img, model, src_fldr, dst_fldr):
    I_org = Image.open(img).convert('RGB')
    
    s_ = 512
    d_ = 128
    
    w, h = I_org.size
    I = np.uint8(I_org).copy()
    I = np.pad(I, ((0, int(s_*np.ceil(I.shape[0]/s_) - I.shape[0])), 
                   (0, int(s_*np.ceil(I.shape[1]/s_) - I.shape[1])), 
                   (0, 0)), mode = "reflect")
    I = Image.fromarray(I)
    
    
    I1 = np.float32(I)#[:256, :256, :]
    I1 = np.transpose(I1, [2, 0, 1])
    Iout = np.zeros_like(I1)
    for i in range(0, I1.shape[1], s_):
        for j in range(0, I1.shape[2], s_):
            It = torch.from_numpy(np.expand_dims(I1[:, i:i+s_, j:j+s_], 0))/255.0
            X = model(It.cuda())
            X = X.data.squeeze().cpu().numpy()
            Iout[:, i:i+s_, j:j+s_] = np.clip(X, 0, 1)
            
    Iout = np.uint8(255 * Iout.transpose([1, 2, 0]))
    Iout = Image.fromarray(Iout).crop((0, 0, w, h))
    
    Iout.save(img.replace(src_fldr, dst_fldr))
    return Iout

def compress_and_decompress2(img, model):
    I_org = Image.open(img).convert('RGB')
    
    s_ = 512
    d_ = 128
    
    w, h = I_org.size
    I = np.uint8(I_org).copy()
    I = np.pad(I, ((0, int(s_*np.ceil(I.shape[0]/s_) - I.shape[0])), 
                   (0, int(s_*np.ceil(I.shape[1]/s_) - I.shape[1])), 
                   (0, 0)), mode = "reflect")
    I = Image.fromarray(I)
    
    
    I1 = np.float32(I)#[:256, :256, :]
    I1 = np.transpose(I1, [2, 0, 1])
    Iout = np.zeros_like(I1)
    for i in range(0, I1.shape[1], s_):
        for j in range(0, I1.shape[2], s_):
            It = torch.from_numpy(np.expand_dims(I1[:, i:i+s_, j:j+s_], 0))/255.0
            X = model(It.cuda())
            X = X.data.squeeze().cpu().numpy()
            Iout[:, i:i+s_, j:j+s_] = np.clip(X, 0, 1)
            
    Iout = np.uint8(255 * Iout.transpose([1, 2, 0]))
    Iout = Image.fromarray(Iout).crop((0, 0, w, h))
    
    
    return Iout

def compress2(img, model):
    I_org = Image.open(img).convert('RGB')
    
    s_ = 512
    d_ = 128
    
    w, h = I_org.size
    I = np.uint8(I_org).copy()
    I = np.pad(I, ((0, int(s_*np.ceil(I.shape[0]/s_) - I.shape[0])), 
                   (0, int(s_*np.ceil(I.shape[1]/s_) - I.shape[1])), 
                   (0, 0)), mode = "reflect")
    I = Image.fromarray(I)
    
    
    I1 = np.float32(I)#[:256, :256, :]
    I1 = np.transpose(I1, [2, 0, 1])
    Eout = []
    for i in range(0, I1.shape[1], s_):
        for j in range(0, I1.shape[2], s_):
            It = torch.from_numpy(np.expand_dims(I1[:, i:i+s_, j:j+s_], 0))/255.0
            X = model.encode(It.cuda())
            X = X.data.squeeze().cpu().numpy()
            Eout.append(X)
            
    return np.concatenate(Eout)

In [4]:
def perform_compression(I_org, model):
    
    w, h = I_org.size
    I = np.uint8(I_org).copy()
    I = np.pad(I, ((0, int(256*np.ceil(I.shape[0]/256) - I.shape[0])), 
                   (0, int(256*np.ceil(I.shape[1]/256) - I.shape[1])), 
                   (0, 0)), mode = "reflect")
    I = Image.fromarray(I)
    
    
    s_ = 512
    d_ = 512//16
    
    c_ = s_//d_
    
    I1 = np.float32(I)#[:256, :256, :]
    I1 = np.transpose(I1, [2, 0, 1])
    Iout = np.zeros_like(I1)
    Enout = np.zeros((3, I1.shape[1]//c_, I1.shape[2]//c_))
    for i in range(0, I1.shape[1], s_):
        for j in range(0, I1.shape[2], s_):
            It = torch.from_numpy(np.expand_dims(I1[:, i:i+s_, j:j+s_], 0))/255.0
            Xe = model.encode(It.cuda())
            X = model.decode(Xe)
            X = X.data.squeeze().cpu().numpy()
            Iout[:, i:i+s_, j:j+s_] = np.clip(X, 0, 1)
            Xe = (Xe + 1)/2
            Enout[:, i//c_:(i+s_)//c_, j//c_:(j+s_)//c_] = Xe.data.squeeze().cpu().numpy()
    
    
    Iout = np.uint8(255 * Iout.transpose([1, 2, 0]))
    Enout = np.uint8(255 * Enout.transpose([1, 2, 0]))
    
    Iout = Image.fromarray(Iout).crop((0, 0, w, h))
    Enout = Image.fromarray(Enout)
    
    psnr = compare_psnr(np.uint8(I_org).copy(), np.uint8(Iout).copy())
#     ssim = compare_ssim(np.uint8(I_org).copy(), np.uint8(Iout).copy())
    
    print('PSNR-PROP :: ' + "{0:0.02f}".format(psnr))
#     print('SSIM-PROP :: ' + "{0:0.02f}".format(ssim), multichannel=True)
    
    image_quality_comp(I_org, Iout, Enout)
    psnr = compare_psnr(np.uint8(I_org).copy(), np.uint8(Image.open('test_10.jpg')).copy())
    print('PSNR-JPG10 :: ' + "{0:0.02f}".format(psnr))
    
    
    return Iout, Enout

In [5]:
def compress(I_org, model):

    e_ = 512
    c_ = 4
    d_ = e_ // c_
    pad_ = 4

    w, h = I_org.size

    comp_w_new = np.ceil(w/c_)
    comp_h_new = np.ceil(h/c_)

    new_w = int(e_ * np.ceil(w/e_))
    new_h = int(e_ * np.ceil(h/e_))

    com_w = new_w // c_
    com_h = new_h // c_

    I = np.uint8(I_org).copy()
    I = np.pad(I, ((0, int(new_h - h)),
                   (0, int(new_w - w)),
                   (0, 0)), mode = "reflect")
    I = Image.fromarray(I)


    I1 = np.float32(I)/255.0
    I1 = np.transpose(I1, [2, 0, 1])

    Enout = np.zeros((3, com_h, com_w))
    Enout_w = np.zeros((3, com_h, com_w))
    for i in list(np.arange(0, new_h, e_)):
        for j in list(np.arange(0, new_w, e_)):
            if i == 0:
                x1 = int(i)
                x2 = int((i + e_) + (pad_*2*c_))
            else:
                x1 = int(i - (pad_*c_))
                x2 = int((i + e_) + (pad_*c_))

            if j == 0:
                y1 = int(j)
                y2 = int((j + e_) + (pad_*2*c_))
            else:
                y1 = int(j - (pad_*c_))
                y2 = int((j + e_) + (pad_*c_))
            It = torch.from_numpy(np.expand_dims(I1[:, x1:x2, y1:y2], 0))
            Xe = model.encode(It.cuda())
            Xe = (Xe + 1.0)/2.0
            Enout[:, x1//c_:x2//c_, y1//c_:y2//c_] += Xe.data.squeeze().cpu().numpy()
            Enout_w[:, x1//c_:x2//c_, y1//c_:y2//c_] += 1.0

    Enout = Enout/Enout_w
    Enout = np.uint8(255 * Enout.transpose([1, 2, 0]))

    Enout = Image.fromarray(Enout).crop((0, 0, comp_w_new, comp_h_new))

    return Enout


def decompress(EnIn, model, w, h):

    e_ = 256
    c_ = 4
    d_ = e_ // c_
    pad_ = 4

#     w, h = int(EnIn.text['w']), int(EnIn.text['h'])

    comp_w_new = np.ceil(w/c_)
    comp_h_new = np.ceil(h/c_)

    new_w = int(e_ * np.ceil(w/e_))
    new_h = int(e_ * np.ceil(h/e_))

    com_w = new_w // c_
    com_h = new_h // c_


    Iout = np.zeros((3,new_h,new_w), dtype = np.float32)
    Iout_w = np.zeros((3,new_h,new_w), dtype = np.float32)

    EnIn = np.uint8(EnIn).copy()
    EnIn = np.pad(EnIn, ((0, int(com_h - EnIn.shape[0])),
                         (0, int(com_w - EnIn.shape[1])),
                         (0, 0)), mode = "reflect")


    EnIn = np.float32(EnIn)/255.0
    EnIn = np.transpose(EnIn, [2, 0, 1])
    for i in list(np.arange(0, com_h, d_)):
        for j in list(np.arange(0, com_w, d_)):

            if i == 0:
                x1 = int(i)
                x2 = int((i + d_) + pad_*2)
            else:
                x1 = int(i - pad_)
                x2 = int((i + d_) + pad_)

            if j == 0:
                y1 = int(j)
                y2 = int((j + d_) + pad_*2)
            else:
                y1 = int(j - pad_)
                y2 = int((j + d_) + pad_)

            It = torch.from_numpy(np.expand_dims(EnIn[:, x1:x2, y1:y2], 0))
            It = It * 2.0 - 1.0
            Xe = model.decode(It.cuda()).data.squeeze().cpu()

            Iout[:, x1*c_:x2*c_, y1*c_:y2*c_] += np.clip(Xe.numpy(), 0, 1)
            Iout_w[:, x1*c_:x2*c_, y1*c_:y2*c_] += 1.0

    Iout = Iout/Iout_w

    Iout = np.uint8(255 * Iout.transpose([1, 2, 0]))
    Iout = Image.fromarray(Iout).crop((0, 0, w, h))

    return Iout


In [6]:
img_file = "/media/cibitaw1/DATA/super_rez/professional_valid/valid/jared-erondu-21325.png"

In [7]:
I = Image.open(img_file).convert("RGB")

In [8]:
model = models.QuantACTShuffleV6()
check_point_file = "/media/cibitaw1/DATA/SP2020/compressACT/weights/"+\
"QuantACTShuffleV6_exp01/checkpoint.pth.tar"
checkpoint = torch.load(check_point_file)
model.load_state_dict(checkpoint['state_dict'], strict = False)
model.cuda()
model.eval()
print('.')

.


In [ ]:
M = models.QuantACTShuffleV3()
check_point_file = "/media/narsi/LargeData/SP_2020/compressACT/weights/QuantACTShuffleV3_exp02/model_best.pth.tar"
checkpoint = torch.load(check_point_file)
M.load_state_dict(checkpoint['state_dict'], strict = False)

model = models.CleanImg(M)
check_point_file = "/media/narsi/LargeData/SP_2020/compressACT/weights/CleanImg_exp01/model_best.pth.tar"
checkpoint = torch.load(check_point_file)
model.load_state_dict(checkpoint['state_dict'], strict = False)

In [9]:
Iout, Enout = perform_compression(I, model)

PSNR-PROP :: 30.43
Original Image :: 
PNG     :: 12.19391966250917
JPG-10% :: 0.1823585381511372
FLIF    :: 10.78488112160675
Encoded Image :: 
PNG     :: 0.07856061995597946
FLIF    :: 0.07661465975788702
PSNR-JPG10 :: 30.35


In [ ]:
Iout = compress_and_decompress2(img_file, model)
image_quality_comp(I, Iout)
psnr = compare_psnr(np.uint8(I).copy(), np.uint8(Iout).copy())
print('PSNR-PROP :: ' + "{0:0.02f}".format(psnr))
psnr = compare_psnr(np.uint8(I).copy(), np.uint8(Image.open('test_10.jpg')).copy())
print('PSNR-JPG35 :: ' + "{0:0.02f}".format(psnr))

In [ ]:
eout = compress2(img_file, model)
eout = np.uint8(eout == 1).reshape(-1)

In [ ]:
len(eout)/(I.size[0] * I.size[1])

In [ ]:
dist = plt.hist(eout, bins = 2)
print(dist[0]/np.sum(dist[0]))

In [ ]:
from range_coder import RangeEncoder, RangeDecoder, prob_to_cum_freq

data = list(eout.tolist())
prob = list(dist[0]/np.sum(dist[0]))

# convert probabilities to cumulative integer frequency table
cumFreq = prob_to_cum_freq(prob, resolution=128)

filepath = 'test.dat'

# encode data
encoder = RangeEncoder(filepath)
encoder.encode(data, cumFreq)
encoder.close()

# decode data
decoder = RangeDecoder(filepath)
dataRec = decoder.decode(len(data), cumFreq)
decoder.close()


In [ ]:
(os.path.getsize('test.dat')*8)/(I.size[0] * I.size[1])

In [25]:
model = models.QuantACTShuffleV6()
model.cuda()
%timeit model.encode(torch.randn(1, 3, 256, 256).cuda())
%timeit model.decode(torch.randn(1, 3, 64, 64).cuda())

2.69 ms ± 40 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.66 ms ± 16.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
model = models.QuantACTShuffleV6()
%timeit model.encode(torch.randn(1, 3, 256, 256))
%timeit model.decode(torch.randn(1, 3, 64, 64))

17 ms ± 494 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
61.8 ms ± 1.09 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
# src_fldr = "/media/cibitaw1/DATA/super_rez/professional_valid/valid"
# imgs = glob(src_fldr + os.sep + "*.png")
src_fldr = "/media/cibitaw1/DATA/super_rez/mobile_valid/valid"
imgs = glob(src_fldr + os.sep + "*.png")
dst_fldr = "/media/cibitaw1/DATA/super_rez/comp_test/decompressed"

In [ ]:
model.cuda()
print('.')

In [ ]:
for img in tqdm(imgs):
    compress_and_decompress(img, model, src_fldr, dst_fldr)

In [14]:
compressed_imgs = [img.replace(src_fldr, dst_fldr) for img in imgs]

In [10]:
from utils.competition_eval import evaluate2

In [15]:
results = evaluate2(compressed_imgs, imgs)

61it [04:37,  3.73s/it]


In [12]:
results

{'PSNR': 29.003067337589883, 'MSSSIM': 0.9377486467526164}

In [16]:
results

{'PSNR': 28.578408862459263, 'MSSSIM': 0.9368299242025995}

In [ ]:
(29.003067337589883+28.83994787307876)/2

In [ ]:
(0.9736846666083886+0.9705319931942877)/2

In [9]:
jpeg2000_comp = "bpgenc"
jpeg2000_decomp = "bpgdec"
src_enco_img = "/home/cibitaw1/local/1WeStar/test_en.png"
src_jp2_img = "/home/cibitaw1/local/1WeStar/test_en_comp.bpg"
src_dejp2_img = "/home/cibitaw1/local/1WeStar/test_en_decomp.png"

In [10]:
Enout = compress(I, model)
metadata = PngInfo()
metadata.add_text("w", str(I.size[0]))
metadata.add_text("h", str(I.size[1]))
Enout.save("test_en.png", pnginfo=metadata)
Enout = Image.open("test_en.png")

# Iout = decompress(Enout, model)

In [12]:
compress_ = []
for i in [25, 29]:
    os.system(jpeg2000_comp + " -m 9 -f 444 -q " + str(i) + " " + src_enco_img + " -o " + src_jp2_img)#-i + " -r " + str(i) + " -b  64,64"
    #-q " + str(i) + " -m 6
    os.system(jpeg2000_decomp + " " + src_jp2_img + " -o " + src_dejp2_img)#-i 
    Enout_ = Image.open(src_dejp2_img)
    Iout = decompress(Enout_, model, I.size[0], I.size[1])
    p_ = compare_psnr(np.uint8(I).copy(), np.uint8(Iout).copy())
    bpp_ = str(8*os.path.getsize(src_jp2_img)/(I.size[0] * I.size[1]))
    compress_.append([i, p_, bpp_])
    print([i, p_, bpp_])
    Iout.save("test_q_" + str(i) + ".png")

[25, 33.472688064350734, '0.11261446886446887']
[29, 32.367402798669254, '0.06944826007326008']


In [13]:
import sewar

In [14]:
for i in [25, 29]:
    print(sewar.msssim(np.uint8(I).copy(), np.uint8(Image.open("test_q_" + str(i) + ".png")).copy()))

(0.9530768884465469+0j)
(0.9303436753855934+0j)
